## 01. `__new__` and `__init__`

[Ref](https://dev.to/delta456/python-init-is-not-a-constructor-12on)

* The `__new__` method is the constructor (it returns the new instance) while `__init__` is just a initializer (the instance is already created when `__init__` is called)

In [1]:
class Point():
    
    def __new__(cls, *args, **kwargs):
        
        print("From new\n")
        print(cls)
        print(args)
        print(kwargs)
        print()
        
        return super().__new__(cls)  # create our object and return it
    
    def __init__(self, x = 0, y = 0):
        
        print("From init\n")
        
        print(self)
        
        self.x = x
        self.y = y

In [2]:
p = Point(3, 4)

From new

<class '__main__.Point'>
(3, 4)
{}

From init



In [3]:
p = Point.__new__(Point, 3, 4)
if isinstance(p, Point):
    type(p).__init__(p, 3, 4)

From new

<class '__main__.Point'>
(3, 4)
{}

From init



#### `__new__`를 사용해 생성할 객체 수를 제한하기

In [4]:
class RectPoint():
    
    max_instance = 4
    instance_created = 0
    
    def __new__(cls, *args, **kwargs):
        
        if cls.instance_created >= cls.max_instance:
            raise ValueError("Cannot create more objects")
            
        cls.instance_created += 1
        
        return super().__new__(cls)
    
    def __init__(self, x = 0, y = 0):
               
        self.x = x
        self.y = y    

In [5]:
p1 = RectPoint(0, 0)
p2 = RectPoint(0, 1)
p3 = RectPoint(1, 0)
p4 = RectPoint(1, 1)

In [6]:
# p5 = RectPoint(2, 2) # ValueError

#### A Simple Singleton 

In [7]:
_singleton = None

class Example:
    
    def __new__(cls):
        
        global _singleton

        if _singleton is None:
            _singleton = super().__new__(cls)

        return _singleton

In [8]:
a = Example()
b = Example()

In [9]:
a is b

True

#### In the following, `int.__init__()` will be called, as the returned object is not of type `Example` but `int`

In [10]:
class Example:   
    def __new__(cls):
        return 3

In [11]:
type(Example())

int

In [12]:
%reset -f

---

## 02. metaclass - `type`

In [13]:
class A:
    pass

print(f'{type(12)      = }')
print(f'{type("Hello") = }')
print(f'{type([])      = }')
print(f'{type(A())     = }')

type(12)      = <class 'int'>
type("Hello") = <class 'str'>
type([])      = <class 'list'>
type(A())     = <class '__main__.A'>


In [14]:
print(f'{type(int)  = }')
print(f'{type(str)  = }')
print(f'{type(list) = }')
print(f'{type(A)    = }')

type(int)  = <class 'type'>
type(str)  = <class 'type'>
type(list) = <class 'type'>
type(A)    = <class 'type'>


![instance_of](./figs/instance-of.png)

In [15]:
B = type('B', (), {})  # https://www.geeksforgeeks.org/python-type-function/
print(f'{B = }')

B = <class '__main__.B'>


In [16]:
b = B()
print(f'{b = }')

b = <__main__.B object at 0x7f7cb00b5be0>


In [17]:
isinstance(b, B)

True

In [18]:
isinstance(B, type)

True

---

* https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/

![instance-creation](./figs/instance-creation.png)
![class-creation](./figs/class-creation.png)

In [19]:
class A1:
    
    a = 1
    b = 'Hello'
    
    def f(self):
        return 42

In [20]:
def make_A2():
    
    name = 'A2'
    bases = ()
    namespace = type.__prepare__(name, bases)
    body = (
"""
a = 1
b = 'Hello'
    
def f(self):
    return 42    
"""
)   
    
    #exec(body, globals(), namespace)
    exec(body, {}, namespace)
    
    A2 = type(name, bases, namespace)
    
    return A2

In [21]:
a1 = A1()

A2 = make_A2()
a2 = A2()

In [22]:
a1.a, a1.b, a1.f() 

(1, 'Hello', 42)

In [23]:
a2.a, a2.b, a2.f()

(1, 'Hello', 42)

---

In [24]:
class MyMetaclass(type): 
    pass

class A(metaclass=MyMetaclass):
    pass

In [25]:
a = A()

print(f'{type(a) = }')
print(f'{type(A) = }')

type(a) = <class '__main__.A'>
type(A) = <class '__main__.MyMetaclass'>


In [26]:
%reset -f

## 03. Method Chaining Classes in Python

In [27]:
s = 'Blessed are the poor in spirit, for theirs is the kingdom of heaven.'

In [28]:
t = s.strip().upper().center(80)
t

'      BLESSED ARE THE POOR IN SPIRIT, FOR THEIRS IS THE KINGDOM OF HEAVEN.      '

---

In [29]:
import numpy as np


class Player:
    def __init__(self, name, position, fatigue=0):
        self.name = name
        self.position = position
        self.fatigue = fatigue

    def draw(self):
        print(f"drawing {self.name} to screen at {self.position}")
        return self  #***********

    def move(self, delta):
        self.position += delta
        self.fatigue += 1
        return self  #*********** 

    def rest(self):
        self.fatigue = 0
        return self  #***********

In [30]:
inha = Player('Inha', np.array([0.0, 0.0]))

UP = np.array([0.0, 1.0])
RIGHT = np.array([1.0, 0.0])

(
  inha.move(UP)
      .move(RIGHT)
      .move(UP)
      .rest()
      .draw()
)

drawing Inha to screen at [1. 2.]


---

* `dataclasses` 모듈은` __init__()` 나 `__repr__()` 과 같은 특수 메서드를 
  사용자 정의 클래스에 자동으로 추가하는 데코레이터와 함수를 제공
  
  [Ref](https://docs.python.org/ko/3/library/dataclasses.html)

In [31]:
from dataclasses import dataclass
import numpy as np

In [32]:
@dataclass(frozen=True)  # slots = True 는 3.10에 추가됨
class Vector:
    x: float
    y: float
    z: float

    def normalized(self):
        x, y, z = self.x, self.y, self.z
        norm = np.sqrt(x*x + y*y + z*z)
        return type(self)(x/norm, y/norm, z/norm)

    def reflected(self):
        return type(self)(-self.x, -self.y, -self.z)

In [33]:
p = Vector(1., 2., 3.)

q = p.reflected().normalized()

print(p)
print(q)

Vector(x=1.0, y=2.0, z=3.0)
Vector(x=-0.2672612419124244, y=-0.5345224838248488, z=-0.8017837257372732)


In [34]:
%reset -f

## 04. Positional-only and keyword-only arguments in Python

In [35]:
import sys
assert sys.version_info >= (3, 8), "positional-only arguments are a Python 3.8+ feature, upgrade your Python!"

#### Either way works

In [36]:
def f(a, b, c):
    print(f'{a = }, {b = }, {c = }')

In [37]:
f(1, 2, 3)
f(a=1, b=2, c=3)
f(c=3, a=1, b=2)
f(1, c=3, b=2)

a = 1, b = 2, c = 3
a = 1, b = 2, c = 3
a = 1, b = 2, c = 3
a = 1, b = 2, c = 3


In [38]:
# f(c=3, 1, b=2) # SyntaxError

#### Force keyword argument

In [39]:
def g(a, b, *, kw_only):
    print(f'{a = }, {b = }, {kw_only = }')

In [40]:
g(1, b=2, kw_only=3)
g(a=1, b=2, kw_only=3)
g(kw_only=3, a=1, b=2)

a = 1, b = 2, kw_only = 3
a = 1, b = 2, kw_only = 3
a = 1, b = 2, kw_only = 3


In [41]:
# g(1, 2) # TypeError
# g(1, 2, 3) # TypeError
# g(b=2, 1, kw_only=3) # SyntaxError

---

In [42]:
def g(a, b, *, kw_only=None):
    print(f'{a = }, {b = }, {kw_only = }')

In [43]:
g(1, b=2, kw_only=3)
g(1, 2)
# g(1, 2, 'oops', kw_only=3) # TypeError

a = 1, b = 2, kw_only = 3
a = 1, b = 2, kw_only = None


In [44]:
def g(a, b, *args, kw_only=None):
    print(f'{a = }, {b = }, {kw_only = }')
    print(f'{args = }')

In [45]:
g(1, 2, 'oops', kw_only=3) # Using * better than *args

a = 1, b = 2, kw_only = 3
args = ('oops',)


---

In [46]:
def eat_args(*args):
    print(args, "yum")
    
def eat_kwargs(**kwargs):
    print(kwargs, "yum!")

In [47]:
eat_args(1, 2, 3)
eat_kwargs(a=1, b=2)

(1, 2, 3) yum
{'a': 1, 'b': 2} yum!


In [48]:
# eat_args(kw=3)  # TypeError
# eat_kwargs(1, 2)  # TypeError

---

In [49]:
def p(a, b, c, /):
    print(f'{a = }, {b = }, {c = }')

In [50]:
p(1, 2, 3)

a = 1, b = 2, c = 3


In [51]:
# p(1, 2, 3, 4) # TypeError
# p(1, 2, c=2) # TypeError

---

In [52]:
def k(*, a, b, c):
    print(f'{a = }, {b = }, {c = }')    

In [53]:
k(a=1, b=2, c=3)

a = 1, b = 2, c = 3


In [54]:
# k(1, 2, 3) # TypeError
# k(1, b=1, c=1) # TypeError

---

In [55]:
def pk1(a, b, k=None):
    print(f'{a = }, {b = }, {k = }')      

In [56]:
pk1(1, 2, k=3)
pk1(1, 2, 3)

a = 1, b = 2, k = 3
a = 1, b = 2, k = 3


In [57]:
def pk1(a, b, *, k=None):
    print(f'{a = }, {b = }, {k = }')     

In [58]:
pk1(1, 2, k=3)
# pk1(1, 2, 3) # TypeError

a = 1, b = 2, k = 3


---

In [59]:
# position_only_builtin_examples:

d = {"a": 1, "b": 2}

x = d.get("c", "missing")
# y = d.get("c", default="missing")  # Error!!! default is positional only

In [60]:
# kw_only_builtin_examples:

# json.load
def load(fp, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw):
    pass

# os.path.realpath
def realpath(path, *, strict=False):
    pass

# pprint.pprint
def pprint(object, stream=None, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True, underscore_numbers=False):
    pass

---

In [61]:
def f(a, b, /, c, d, *, e, f):
    pass

def f(pos_only, /, pos_or_kw, *, kw_only):
    pass

---

In [62]:
from timeit import timeit

def speed_differences():
    
    trials = 10 ** 7
    per_trials = 10 ** 9 / trials # nanoseconds
    
    def func(a, b, c):
        pass

    t1 = timeit(stmt="func(1, 2, 3)", globals={'func': func}, number=trials) * per_trials
    t2 = timeit(stmt="func(a=1, b=2, c=3)", globals={'func': func}, number=trials) * per_trials
    t3 = timeit(stmt="func(c=3, a=1, b=2)", globals={'func': func}, number=trials) * per_trials
    t4 = timeit(stmt="func(1, c=3, b=2)", globals={'func': func}, number=trials) * per_trials

    def func(a, b, c, /):
        pass

    t5 = timeit(stmt="func(1, 2, 3)", globals={'func': func}, number=trials) * per_trials

    def func(*, a, b, c):
        pass

    t6 = timeit(stmt="func(a=1, b=2, c=3)", globals={'func': func}, number=trials) * per_trials
    t7 = timeit(stmt="func(c=3, b=2, a=1)", globals={'func': func}, number=trials) * per_trials

    print("normal func\n")
    print(f'{t1=:.2f}\t\t func(1, 2, 3)')
    print(f'{t2=:.2f}\t\t func(a=1, b=2, c=3)')
    print(f'{t3=:.2f}\t\t func(c=3, a=1, b=2)')
    print(f'{t4=:.2f}\t\t func(1, c=3, b=2)')
    print()

    print("pos only\n")
    print(f'{t5=:.2f}\t\t func(1, 2, 3)')
    print()

    print("kw only\n")
    print(f'{t6=:.2f}\t\t func(a=1, b=2, c=3)')
    print(f'{t7=:.2f}\t\t func(c=3, b=2, a=1)')

In [63]:
speed_differences()

normal func

t1=77.15		 func(1, 2, 3)
t2=88.69		 func(a=1, b=2, c=3)
t3=88.43		 func(c=3, a=1, b=2)
t4=86.57		 func(1, c=3, b=2)

pos only

t5=76.65		 func(1, 2, 3)

kw only

t6=86.95		 func(a=1, b=2, c=3)
t7=87.64		 func(c=3, b=2, a=1)


In [64]:
%reset -f

## 05. `super` in Python

In [65]:
class Base:
    def f(self, x):
        print("Base.f", self, x)


class Derived(Base):
    def f(self, x):
        print("Derived.f", self, x)
        super().f(x)  # don't spefify self
        print("Derived.f finished")

In [66]:
d = Derived()
d.f(42)

Derived.f <__main__.Derived object at 0x7f7cc16ba4c0> 42
Base.f <__main__.Derived object at 0x7f7cc16ba4c0> 42
Derived.f finished


In [67]:
class Base:
    
    @classmethod
    def f(cls, x):
        print("Base.f", cls, x)


class Derived(Base):
    
    @classmethod
    def f(cls, x):
        print("Derived.f", cls, x)
        super().f(x)  # don't spefify cls
        print("Derived.f finished")

In [68]:
d = Derived()
d.f(42)

Derived.f <class '__main__.Derived'> 42
Base.f <class '__main__.Derived'> 42
Derived.f finished


---

In [69]:
class A:
    def f(self):
        print(f"called A.f, self is {self}")

class B(A):
    def f(self):
        print(f"called B.f, self is {self}")

In [70]:
b = B()
sup = super(B, b)

print("super self      : ", sup.__self__)
print("super self class: ", sup.__self_class__)
print("super thisclass : ", sup.__thisclass__)

super self      :  <__main__.B object at 0x7f7cc16bc340>
super self class:  <class '__main__.B'>
super thisclass :  <class '__main__.B'>


In [71]:
sup = super(A, b)

print("super self      : ", sup.__self__)
print("super self class: ", sup.__self_class__)
print("super thisclass : ", sup.__thisclass__)

super self      :  <__main__.B object at 0x7f7cc16bc340>
super self class:  <class '__main__.B'>
super thisclass :  <class '__main__.A'>


---

In [72]:
class Root:
    def f(self):
        print("Root.f", self)

class A(Root):
    pass

class B(A):
    def f(self):
        print("B.f", self)
        super().f()

b = B()
b.f()

B.f <__main__.B object at 0x7f7cc16bacd0>
Root.f <__main__.B object at 0x7f7cc16bacd0>


In [73]:
class Root:
    
    def f(self):
        print("Root.f", self)
        assert not hasattr(super(), 'f'), "You forgot to inherit from Root"

class A(Root):
    def f(self):
        print("A.f", self)
        super().f()

class B(Root):
    def f(self):
        print("B.f", self)
        super().f()

class C(A, B):
    def f(self):
        print("C.f", self)
        super().f()

C().f()

C.f <__main__.C object at 0x7f7cc16cf460>
A.f <__main__.C object at 0x7f7cc16cf460>
B.f <__main__.C object at 0x7f7cc16cf460>
Root.f <__main__.C object at 0x7f7cc16cf460>


In [74]:
class Root:
    pass

class A(Root):
    pass

class B(Root):
    pass

class C(A, B):
    pass

print([cls.__name__ for cls in C.__mro__])

['C', 'A', 'B', 'Root', 'object']


In [75]:
class A:        # (A, object)
    pass

class B:        # (B, object)
    pass

class C(A, B):  # (C, A, B, object)
    pass        #    (A,    object)
                #       (B, object)

# class D(A, C):  # TypeError: Cannot create a consistent method resolution order (MRO) for bases A, C
#     pass

In [76]:
%reset -f